# SVD Classifier

In [21]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
import time
import random
#!pip install ProgressBar
from progressbar import ProgressBar
import math
#!{sys.executable} -m pip install ProgressBar
#!pip install ProgressBar
from random import randrange
from copy import copy, deepcopy

import glob
import json

#svd stuff
from scipy.linalg import svd
from numpy.linalg import pinv


In [298]:
#load the titles and wheater the book is alive or dead into two seperate arrays
def loadResult():
    test_labels = []
    test_values = []
    with open('table-of-contents.json', encoding="utf8") as f:
        data = json.load(f)
        for books in data:
            for book in data[books]:
                test_labels.append(book['title'])
                test_values.append(book['dead'])
                
    return test_labels, test_values

#get the data from the data output file
def loadData():
    final_data = []
    with open('dataOutput2.json', encoding="utf8") as f:
        data = json.load(f)
        for index in data: 
            final_data.append(data[index]['chapData'])
    return final_data

In [311]:

                

title_labels, dead_labels = loadResult()
train = loadData()
print(len(train))
## Load the training set
train_data= np.array(train[:15705])
train_labels = np.array(dead_labels[:15705])
train_titles = np.array(title_labels[:15705])


## Load the testing set

test_data= np.array(train[-1000:]) 
test_labels = np.array(dead_labels[-1000:])
test_titles = np.array(title_labels[-1000:])


## Print out their dimensions
print("Training dataset dimensions: ", np.shape(train_data))
print("Number of training labels: ", len(train_labels))
print("Testing dataset dimensions: ", np.shape(test_data))
print("Number of testing labels: ", len(test_labels))



16705
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    1    1    0    0
    0    0    0    0    1    0    0    0    0    0    0    0    0    0
    0    0    0    1    1    0    0    0    0 2547  849    2    0    1
   36]
[    1     0     0     0     0     0     0     0     0     1     0     1
     0     0     0     0     0     0     0     0     1     0     0     0
     0     0     0     0     1     0     0     0     0     0     0     0
     0     0     1     0     0     0     0     1     0     0     0     0
     0     0     0     0     0     1     0     1     1     0     0     1
     1     0     0     0     0 10185 10185   445    46   119     5]
Training dataset dimensions:  (15705, 71)
Number of training labels:  15705
Testing dataset dimensions:  (1000, 71)
Number of testing labels:  1000


For the next two items we are attempting to recreating the element. A should equal B


calculate pseudo inverse manually and compare to the built in version

In [300]:
#A= np.array([[0.1, 0.2],[0.3, 0.4],[0.5, 0.6],[0.7, 0.8]])
A=np.array(train_data[:])
print(A)
# calculate pseudoinverse
B = pinv(A)
print(B)

[[  0   0   0 ...   0   1  36]
 [  0   0   0 ...   0   1   7]
 [  0   0   0 ...  20  25 689]
 ...
 [  0   0   0 ...   1   0   5]
 [  0   0   0 ...   2   0   1]
 [  0   0   1 ... 187 255 232]]
[[ 1.41665154e-06 -7.87624385e-05 -1.50063288e-05 ... -8.81508881e-05
  -2.43381414e-04 -1.57186015e-04]
 [-3.11887480e-05  4.14462675e-05 -2.39004888e-04 ... -1.38122724e-04
  -1.99278900e-05 -3.38553861e-04]
 [-2.30330965e-05 -3.87923977e-05 -8.42779723e-05 ... -1.93302046e-04
  -1.29650184e-05  1.44900618e-03]
 ...
 [-3.53724560e-07  4.86100109e-07  1.05452792e-06 ...  6.99338964e-07
   5.91041688e-07 -3.66002797e-06]
 [ 4.49952314e-07 -4.67608076e-07 -1.13338360e-07 ... -6.02837625e-07
  -5.32801834e-07  3.21050947e-06]
 [-3.71539162e-09 -5.57172465e-08  5.07890139e-07 ... -1.54623509e-07
  -2.89767954e-08 -2.49413822e-07]]


In [301]:
print(A)
# calculate svd
U, s, VT = svd(A)
# reciprocals of s
d = 1.0 / s
# create m x n D matrix
D = np.zeros(A.shape)
# populate D with n x n diagonal matrix
D[:A.shape[1], :A.shape[1]] = np.diag(d)
# calculate pseudoinverse
B = VT.T.dot(D.T).dot(U.T)
print(B)


[[  0   0   0 ...   0   1  36]
 [  0   0   0 ...   0   1   7]
 [  0   0   0 ...  20  25 689]
 ...
 [  0   0   0 ...   1   0   5]
 [  0   0   0 ...   2   0   1]
 [  0   0   1 ... 187 255 232]]
[[ 1.41665154e-06 -7.87624385e-05 -1.50063288e-05 ... -8.81508881e-05
  -2.43381414e-04 -1.57186015e-04]
 [-3.11883520e-05  4.14462206e-05 -2.39004961e-04 ... -1.38122710e-04
  -1.99278888e-05 -3.38553853e-04]
 [-2.30322454e-05 -3.87925141e-05 -8.42781799e-05 ... -1.93302006e-04
  -1.29647602e-05  1.44900626e-03]
 ...
 [-3.53724719e-07  4.86100125e-07  1.05452794e-06 ...  6.99338961e-07
   5.91041740e-07 -3.66002796e-06]
 [ 4.49952818e-07 -4.67608132e-07 -1.13338439e-07 ... -6.02837610e-07
  -5.32801893e-07  3.21050947e-06]
 [-3.71532570e-09 -5.57172542e-08  5.07890127e-07 ... -1.54623507e-07
  -2.89767968e-08 -2.49413821e-07]]


In [302]:
def getError(B, data, labels):

    true = 0

    for i in range(len(data)):
        x = data[i]
        predict = np.sum(x.dot(B))

        #find the error
        if (labels[i] == False) and (predict<1):
            true +=1
        elif (labels[i] == True) and (predict>1):
            true+=1



        #these are  the results from the svd classification
        #print(predict, end = ": ")
        #print(train_labels[i])

        #calculate the error

    #print('error rate: ', 1-(true/len(train_data)))
    return 1-(true/len(data))

In [303]:
print('error rate: ', getError(B, train_data, train_labels))
print('test rate ', getError(B, test_data, test_labels))

error rate:  0.3444126074498567
test rate  0.32699999999999996


# Compute the derivative


We will now compute the derivative of the regression cost function: 
$$L_D(w) = \frac{1}{n}\sum_{i=1}^n (y_i-w\cdot x_i)^2,$$
where $x_i\in \mathrm{R}^d$ is the input feature of dimension $d$, $y_i\in\mathrm{R}$ is the output response, and $w\in\mathrm{R}^d$ is the regression weights.

**Task P3:** Complete the function 'weight_derivative' to calculate the derivative of the cost function with respect to regression weights $w$, i.e., $\frac{\partial}{\partial w}L_D(w)$. Note that this should be a $d$ dimensional vector. Also copy the output of the code for the test example to the solution file.

In [304]:
def weight_derivative(weights, feature_matrix, labels):
    # Input:
    # weights: weight vector w, a numpy vector of dimension d
    # feature_matrix: numpy array of size n by d, where n is the number of data points, and d is the feature dimension
    # labels: true labels y, a numpy vector of dimension d
    # Output:
    # Derivative of the regression cost function with respect to the weight w, a numpy array of dimension d
        
    ## STUDENT: Start of code ###
    
    test_predictions = np.dot(weights,feature_matrix)
    
    
    
    errors = test_predictions - labels
    
    derivative = np.dot(feature_matrix,np.transpose(errors))
    derivative = derivative/(np.sum(derivative))
    return(derivative)
    # End of code ###

In [305]:
# NOTE: copy the output to the solution file.



my_weights = np.ones(len(train_data[0])) # this makes all the predictions 0
derivative = weight_derivative(my_weights, B, train_labels)

print (derivative)

[ 4.51555872e-20 -9.61713742e-17 -5.92519013e-16 -1.28320228e-16
  6.13282822e-17 -3.27404760e-16 -6.71135514e-17 -2.62725546e-17
  2.96568709e-16 -4.83750358e-17  9.26317472e-17  2.38855197e-16
 -1.73723779e-16 -2.66681840e-17 -9.98688248e-17 -3.34050930e-16
 -2.85039114e-16  5.00000301e-01 -8.11113977e-16 -3.20325316e-16
 -2.43297182e-16  2.84696476e-16  1.78917409e-16 -1.37586376e-18
 -2.33402190e-16  5.90503838e-16 -3.76530538e-16 -5.97658498e-16
  6.23338534e-17  2.16237201e-16  6.39826970e-16 -4.05688942e-17
  2.31526933e-16 -4.10611312e-16 -3.75145921e-16 -6.39388390e-16
  7.02236553e-16 -4.19627134e-16 -1.57378623e-17 -1.57886652e-16
  1.28766053e-16 -4.41106025e-16  1.72990441e-16 -5.45188641e-17
 -1.44584219e-16 -1.16906387e-18 -9.69117582e-17  8.52081894e-17
 -1.15959261e-16  4.14856549e-16  2.35172768e-16  4.99999699e-01
  2.70075316e-16  2.81446554e-16 -7.48279612e-18 -9.60986243e-17
 -1.59045778e-16 -3.96531751e-16  2.71607571e-16  5.36385081e-16
 -3.54320512e-16 -5.35683

Here, we will write a function to perform gradient descent algorithm on the lineare regression cost. Given an initial point, we will update the current weights by moving in the negative gradient direction to minimize the cost function. Thus, in each iteration we obtain the updated weight $w_{t+1}$ from the current iterate $w_t$ as follows:
$$w_{t+1} = w_t - h\frac{\partial}{\partial w}L_D(w_t),$$
where $h$ is the 'step_size' that is the amount by which we move in the negative gradient direction. 

We stop when we are sufficiently close to the optimum (where gradient is the zero vector) by checking the condition with respect to the magnitude (length) of the gradient vector:
$$\|\frac{\partial}{\partial w}L_D(w_t)\|_2\leq \epsilon,$$
where $\epsilon$ is the 'tolerance' parameter.

**Task P4:** Complete the code section to perform the gradient decent in the function `regression_gradient_descent`. Copy the code to the solution file.

In [306]:
def regression_gradient_descent(feature_matrix, labels, initial_weights, step_size, tolerance):
    # Gradient descent algorithm for linear regression problem    
    
    # Input:
    # feature_matrix: numpy array of size n by d, where n is the number of data points, and d is the feature dimension
    # labels: true labels y, a numpy vector of dimension d
    # initial_weights: initial weight vector to start with, a numpy vector of dimension d
    # step_size: step size of update
    # tolerance: tolerace epsilon for stopping condition
    # Output:
    # Weights obtained after convergence
    
    converged = False 
    weights = np.array(initial_weights) # current iterate
    i = 0
    while not converged:
        # Start of code: your impelementation of what the gradient descent algorithm does in every iteration
        # Refer back to the update rule listed above: update the weight
        i += 1
        derivative = weight_derivative(weights, feature_matrix, labels)
        #print(derivative)
        weights -= (step_size * derivative)
        
        # Compute the gradient magnitude:
        
        gradient_magnitude = np.sqrt(np.sum(derivative**2))
        
        # Check the stopping condition to decide whether you want to stop the iterations
        #print ("grad mag :", gradient_magnitude)
        #print ("tolerance:", tolerance)
        if gradient_magnitude > tolerance:
            converged = True
        
        # End of code
        
        print ("Iteration: ",i,"gradient_magnitude: ", gradient_magnitude) # for us to check about convergence
        
    return weights

In [309]:
#STUDENT: Specify the initial_weights, step_size, and tolerance
simple_feature_matrix = B
output = train_labels



temp_weights = [np.ones(len(train_data[0]))]*(len(train_data[0]))# this makes all the predictions 0 #np.array([0.]*2*len(simple_feature_matrix))
temp_weights=np.array(temp_weights)
initial_weights = temp_weights.reshape(len(simple_feature_matrix),len(train_data[0]))

step_size = 100
tolerance = 0.1
# end of code

# Use the regression_gradient_descent function to calculate the gradient decent and store it in the variable 'final_weights'
final_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

# end of code
print ("Here are the final weights after convergence:")

print (final_weights)

Iteration:  1 gradient_magnitude:  0.08391813582968449
Iteration:  2 gradient_magnitude:  0.08450900179224057
Iteration:  3 gradient_magnitude:  0.08492922650968128
Iteration:  4 gradient_magnitude:  0.08506941476564205
Iteration:  5 gradient_magnitude:  0.08511000123463833
Iteration:  6 gradient_magnitude:  0.08512133050496345
Iteration:  7 gradient_magnitude:  0.08512446184276319
Iteration:  8 gradient_magnitude:  0.08512532498245
Iteration:  9 gradient_magnitude:  0.08512556272608893
Iteration:  10 gradient_magnitude:  0.08512562819741692
Iteration:  11 gradient_magnitude:  0.085125646227886
Iteration:  12 gradient_magnitude:  0.08512565119802967
Iteration:  13 gradient_magnitude:  0.08512565258211596
Iteration:  14 gradient_magnitude:  0.08512565300935926
Iteration:  15 gradient_magnitude:  0.08512565326423606
Iteration:  16 gradient_magnitude:  0.08512565374269329
Iteration:  17 gradient_magnitude:  0.08512565508897446
Iteration:  18 gradient_magnitude:  0.08512565907246118
Iterat

In [292]:
print(len(B[0]))
print(len(final_weights[0]))

x = np.dot(np.transpose(final_weights),B)

print(len(x))
print('error rate: ', getError(x, train_data, train_labels))
print('test rate: ', getError(x, test_data, test_labels))

15705
69
69
error rate:  0.1217446673034066
test rate:  0.10899999999999999
